In [2]:
import pandas as pd
import pathlib as pl
import plotly.express as px
import numpy as np
import plotly.io as pio
import os
import img2pdf

file = pl.Path("../recieved.csv")

headerNames = ["time", "recieverId", "tempInC", "pressure", "altitude", "humidity", "capacitance"]
dataCols = ["tempInC", "pressure", "altitude", "humidity", "capacitance"]
recievedDf = pd.read_csv(file, names=headerNames)  # dF means DataFrame, a construct in pandas for tabled data

# pd.options.plotting.backend = "plotly"

uniqueIDs = sorted(recievedDf.recieverId.unique())
print(f"IDs Connected: {uniqueIDs}")
figList = []
for uniqueID in uniqueIDs:
    figsOfId = []
    tempDf = recievedDf[recievedDf['recieverId']==uniqueID]
    tempDf = tempDf[np.abs(tempDf.capacitance - tempDf.capacitance.mean()) <= (3 * tempDf.capacitance.std())] #keep only the ones that are within +3 to -3 standard deviations in the column 'Data'.
    tempDf = tempDf[~(np.abs(tempDf.capacitance - tempDf.capacitance.mean()) > (3 * tempDf.capacitance.std()))] #or the other way around
    for col in dataCols:
        tempTitle = (f"DEVICE-{uniqueID}-{col}").upper()
        figsOfId.append(px.scatter(tempDf, x="time", y=col, title=tempTitle, template="plotly_white"))    
    figList.append(figsOfId)

for reciever in figList:
    for plot in reciever:
        place = pl.Path(f"output/images/{plot.layout.title.text}.png")
        plot.write_image(place)


IDs Connected: [1, 2, 3, 4]


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
# Convert Images to one PDF
with open(pl.Path("output/sensorPlots.pdf"), "wb") as f:
    fullPaths = [os.path.join("output/images/", name) for name in os.listdir("output/images/")]
    f.write(img2pdf.convert([i for i in fullPaths if i.endswith(".png")]))

In [ ]:
# MAP Example
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron")
fig.show()